In [2]:
# ==============================
# 1. IMPORT LIBRARIES
# ==============================

import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.calibration import CalibratedClassifierCV

In [23]:
# ==============================
# 2. LOAD DATA
# ==============================

df = pd.read_csv("../data/maharashtra_onion.csv")

df['Date'] = pd.to_datetime(df['Date'])

# Rename columns cleanly
df = df.rename(columns={
    'Arrival Quantity 01-01-2023 to 09-02-2026': 'Arrival_MT',
    'Modal Price 01-01-2023 to 09-02-2026': 'Modal_Price'
})

df = df.sort_values(['District', 'Date'])

# Convert arrival to quintals
df['Arrival_Qtl'] = df['Arrival_MT'] * 10


/var/folders/9b/bmcr0vkn0gvfwsslztlf62f80000gn/T/ipykernel_84403/2449382670.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [4]:
# ---------------------------------
# DISTRICT GEO MAPPING
# ---------------------------------

district_geo = {
    "Nashik": (19.9975, 73.7898),
    "Pune": (18.5204, 73.8567),
    "Dhule": (20.9042, 74.7740),
    "Kolhapur": (16.7050, 74.2433),
    "Satara": (17.6805, 74.0183),
    "Raigad": (18.5158, 73.1822),
    "Sholapur": (17.6599, 75.9064),
    "Amarawati": (20.9320, 77.7523),
    "Chandrapur": (19.9615, 79.2961),
    "Nandurbar": (21.3655, 74.2400),
    "Thane": (19.2183, 72.9781),
    "Dharashiv(Usmanabad)": (18.1860, 76.0419),
    "Sangli": (16.8524, 74.5815),
    "Jalana": (19.8410, 75.8860),
    "Wardha": (20.7453, 78.6022),
    "Akola": (20.7096, 76.9981),
    "Buldhana": (20.5292, 76.1840),
    "Jalgaon": (21.0077, 75.5626),
    "Beed": (18.9901, 75.7531),
    "Nagpur": (21.1458, 79.0882),
    "Mumbai": (19.0760, 72.8777),
    "Ahmednagar": (19.0948, 74.7480),
    "Chattrapati Sambhajinagar": (19.8762, 75.3433)
}


In [6]:
# ---------------------------------
# RAINFALL FETCH FUNCTION (CLEANED)
# ---------------------------------

import requests
import pandas as pd
from datetime import date, timedelta

def fetch_rainfall(lat, lon, days=30):
    end_date = date.today()
    start_date = end_date - timedelta(days=days)

    start_str = start_date.strftime("%Y%m%d")
    end_str = end_date.strftime("%Y%m%d")

    url = (
        "https://power.larc.nasa.gov/api/temporal/daily/point"
        f"?parameters=PRECTOTCORR"
        f"&community=AG"
        f"&longitude={lon}"
        f"&latitude={lat}"
        f"&start={start_str}"
        f"&end={end_str}"
        f"&format=JSON"
    )

    response = requests.get(url)
    data = response.json()

    if "properties" not in data:
        print("API error:", data)
        return None

    rainfall_data = data["properties"]["parameter"]["PRECTOTCORR"]

    df_rain = pd.DataFrame.from_dict(
        rainfall_data,
        orient="index",
        columns=["rainfall_mm"]
    )

    df_rain.index = pd.to_datetime(df_rain.index, format="%Y%m%d")

    # 🔥 CLEANING STEP
    df_rain["rainfall_mm"] = df_rain["rainfall_mm"].replace(-999, pd.NA)
    df_rain["rainfall_mm"] = pd.to_numeric(df_rain["rainfall_mm"], errors="coerce")
    df_rain = df_rain.dropna()

    return df_rain


In [8]:
# Fetch recent 30 days
lat, lon = district_geo["Nashik"]
df_recent = fetch_rainfall(lat, lon, days=30)

# Fetch last 365 days as baseline
df_year = fetch_rainfall(lat, lon, days=365)

# Compute means
recent_mean = df_recent["rainfall_mm"].mean()
baseline_mean = df_year["rainfall_mm"].mean()

rain_anomaly = recent_mean - baseline_mean

print("Recent 30-day mean rainfall:", round(recent_mean, 2))
print("Baseline 365-day mean rainfall:", round(baseline_mean, 2))
print("Rainfall anomaly:", round(rain_anomaly, 2))


Recent 30-day mean rainfall: 0.01
Baseline 365-day mean rainfall: 6.06
Rainfall anomaly: -6.05


In [9]:
# ---------------------------------
# COMPUTE RAINFALL ANOMALY FUNCTION
# ---------------------------------

def compute_rain_anomaly(district_name, recent_days=30, baseline_days=365):
    
    if district_name not in district_geo:
        return None
    
    lat, lon = district_geo[district_name]
    
    df_recent = fetch_rainfall(lat, lon, days=recent_days)
    df_year = fetch_rainfall(lat, lon, days=baseline_days)
    
    if df_recent is None or df_year is None:
        return None
    
    recent_mean = df_recent["rainfall_mm"].mean()
    baseline_mean = df_year["rainfall_mm"].mean()
    
    rain_anomaly = recent_mean - baseline_mean
    
    return rain_anomaly


In [10]:
# ---------------------------------
# BUILD DISTRICT RAINFALL TABLE
# ---------------------------------

rain_data = []

for district in df['District'].unique():
    
    anomaly = compute_rain_anomaly(district)
    
    rain_data.append({
        "District": district,
        "rain_anomaly_30d": anomaly
    })

rain_df = pd.DataFrame(rain_data)

rain_df


,District,rain_anomaly_30d
0,Ahmednagar,-2.126517
1,Akola,-3.117028
2,Amarawati,-2.993467
3,Beed,-2.198050
4,Buldhana,-2.847494
5,Chandrapur,-4.710317
6,Chattrapati Sambhajinagar,-2.406622
7,Dharashiv(Usmanabad),-3.033811
8,Dhule,-1.674822
9,Jalana,-2.406622


In [56]:
# ---------------------------------
# MERGE RAINFALL INTO MAIN DATASET
# ---------------------------------

df = df.merge(rain_df, on="District", how="left")

df[['District', 'rain_anomaly_30d']].head()


,District,rain_anomaly_30d
0,Ahmednagar,-2.114383
1,Ahmednagar,-2.114383
2,Ahmednagar,-2.114383
3,Ahmednagar,-2.114383
4,Ahmednagar,-2.114383


In [11]:
# ==============================
# 3. FEATURE ENGINEERING (ALL DISTRICTS)
# ==============================

df = df.sort_values(['District', 'Date'])

# Group by district to avoid leakage
df['ret_1'] = df.groupby('District')['Modal_Price'].pct_change(1)
df['ret_3'] = df.groupby('District')['Modal_Price'].pct_change(3)
df['ret_7'] = df.groupby('District')['Modal_Price'].pct_change(7)

df['ma_7'] = df.groupby('District')['Modal_Price'].rolling(7).mean().reset_index(level=0, drop=True)
df['vol_14'] = df.groupby('District')['Modal_Price'].pct_change().rolling(14).std().reset_index(level=0, drop=True)

df['arrival_3pct'] = df.groupby('District')['Arrival_Qtl'].pct_change(3)
df['arrival_7mean'] = df.groupby('District')['Arrival_Qtl'].rolling(7).mean().reset_index(level=0, drop=True)

# Arrival spike per district
df['arrival_spike'] = df.groupby('District')['Arrival_Qtl'].transform(
    lambda x: (x > x.quantile(0.90)).astype(int)
)

df['month'] = df['Date'].dt.month

# District encoding
df['district_code'] = df['District'].astype('category').cat.codes

In [58]:
# ==============================
# 4. CRASH LABEL CREATION
# ==============================

horizon = 7
threshold_drop = 0.15

df['future_min_7d'] = df.groupby('District')['Modal_Price'].transform(
    lambda x: x.shift(-1).rolling(horizon).min()
)

df['crash_label_7d'] = (
    df['future_min_7d'] < df['Modal_Price'] * (1 - threshold_drop)
).astype(int)

df = df.dropna()

In [59]:
# ==============================
# 5. FEATURE SELECTION
# ==============================

feature_cols = [
    'ret_1','ret_3','ret_7',
    'ma_7','vol_14',
    'arrival_3pct','arrival_7mean',
    'arrival_spike',
    'month',
    'district_code',
    'rain_anomaly_30d'
]


X = df[feature_cols]
y = df['crash_label_7d']

In [60]:
# ==============================
# 6. TIME SERIES VALIDATION + CLASS WEIGHTING
# ==============================

tscv = TimeSeriesSplit(n_splits=5)

auc_scores = []

for train_idx, test_idx in tscv.split(X):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Compute class weights
    classes = np.unique(y_train)
    weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=y_train
    )

    class_weights = dict(zip(classes, weights))
    scale_pos_weight = class_weights[1] / class_weights[0]

    model = xgb.XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        eval_metric='auc',
        scale_pos_weight=scale_pos_weight
    )

    model.fit(X_train, y_train)

    probs = model.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, probs)
    auc_scores.append(auc)

print("Average Time-Series AUC:", np.mean(auc_scores))



Average Time-Series AUC: 0.9059321646491181


In [61]:
# ==============================
# 7. FINAL TRAIN ON FULL DATA
# ==============================

classes = np.unique(y)
weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y
)

class_weights = dict(zip(classes, weights))
scale_pos_weight = class_weights[1] / class_weights[0]

base_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    eval_metric='auc',
    scale_pos_weight=scale_pos_weight
)

base_model.fit(X, y)

# ==============================
# 8. CALIBRATION
# ==============================

calibrated_model = CalibratedClassifierCV(base_model, method='isotonic', cv=3)
calibrated_model.fit(X, y)



,estimator,"XGBClassifier...ree=None, ...)"
,method,'isotonic'
,cv,3
,n_jobs,None
,ensemble,'auto'
,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None


In [62]:
# ==============================
# 9. SAVE MODEL
# ==============================

joblib.dump(calibrated_model, "../models/xgb_crash_model.joblib")

print("Robust calibrated model saved successfully.")

Robust calibrated model saved successfully.
